# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use Score/Rating. A rating of 4 or 5 can be cosnidered as a positive review. A rating of 1 or 2 can be considered as negative one. A review of rating 3 is considered nuetral and such reviews are ignored from our analysis. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




In [1]:
import nltk
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns              
from nltk import FreqDist
#nltk.download()
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prudi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#import spacy
#import gensim
#from gensim import corpora

In [3]:
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\amazon-fine-food-reviews')
#df = pd.read_json('Reviews.json', lines=True)
df=pd.read_csv('Reviews.csv')

In [4]:
df.loc[df['ProductId'] == 'B000HDOPZG',:]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
73779,73780,B000HDOPZG,ANHNUHKI585T7,L. Hirsch,14,14,5,1278374400,Irresistable!,"My husband and I just looked at each other after eating one of the lemon wafer cookies and sighed. The flavor is pure deep natural lemon, tart and bright, with a sweet creamy finish--nothing like..."
73780,73781,B000HDOPZG,AQFTBJZPVNMAH,Hippo,7,7,4,1254960000,Tasty & Addictive,"It is a very tasty treat I enjoy with my afternoon coffee. However, all good things must be had in moderation, and hence I will only buy one pack at a time from the local store.<br />However the p..."
73781,73782,B000HDOPZG,A2G2DXPFZKG48W,Audio Maven,6,6,5,1204070400,Great tasting snack,These are the greatest wafer type cookies I have ever eaten. I've also tried the lemon ones which are out of this world as well. They are wafer type but are cut into small cubes (ideal for snack s...
73782,73783,B000HDOPZG,AL1DVFEY7X5VR,Always Snackin',5,5,5,1199923200,Can't eat just one!,ALL I CAN SAY IS OMG!THESE LITTLE TREATS ARE SO ADDICTIVE AND ONLY HAVE 10 GRAMS OF SUGAR IN NINE WAFERS!THESE ARE BY FAR THE BEST TASTING WAFER COOKIES I HAVE EVER HAD AND IF IT WERE NOT FOR ME B...
73783,73784,B000HDOPZG,A2F64XFMO5ELL6,Rod B,3,3,4,1260489600,Quadratini Lemon are great!,"My wife loves these little wafers, but our local stores no longer carry them. Amazon to the rescue! Great product, fast shipping. We are very happy and will continue to order these."
73784,73785,B000HDOPZG,A16SDTBEKKFN2Z,M. White,2,2,5,1336521600,delicious,delicious hazel nut taste in a cunchy bite- size wafer cookie. highly recommended. will buy these again.remind me of the hazel nut coffee creamer
73785,73786,B000HDOPZG,AI47WBE7PHESF,J. Collins,2,2,5,1278720000,Punch of lemon flavor,These Loacker Quadratini Lemon Wafers are the best I've tasted. They are natural with a real lemon 'punch' in every bite. Plus they are not overly sweet like some other wafer brands. I will defini...
73786,73787,B000HDOPZG,ALT34F4DUTJ8Z,Sassy08,2,2,5,1278288000,Bet you can't eat just one!,"Previously, these lemon wafer cookies were sold at Wal Mart; however, they no longer carry them. Recently, I discovered that Trader Joe's carried a chocolate version and I inquired if they carrie..."
73787,73788,B000HDOPZG,A3BIN78AJ0D4JO,"MrsDiehl ""MrsDiehl""",2,2,5,1272067200,Special treat,I tried this product at a friend's house and had to get some for myself. I was delighted to find them on Amazon for half the price listed on another so called wholesale site. These make a great ...
73788,73789,B000HDOPZG,AKFGO1HY01B85,L. Botez,2,2,5,1270598400,Yum,I love these cookies! what can I say. They are worth buying in bulk! Yum. Bet you can't eat just one.


In [5]:
df.loc[df['ProductId'] == 'B000HDL1RQ',:]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
78433,78434,B000HDL1RQ,ANHNUHKI585T7,L. Hirsch,14,14,5,1278374400,Irresistable!,"My husband and I just looked at each other after eating one of the lemon wafer cookies and sighed. The flavor is pure deep natural lemon, tart and bright, with a sweet creamy finish--nothing like..."
78434,78435,B000HDL1RQ,AQFTBJZPVNMAH,Hippo,7,7,4,1254960000,Tasty & Addictive,"It is a very tasty treat I enjoy with my afternoon coffee. However, all good things must be had in moderation, and hence I will only buy one pack at a time from the local store.<br />However the p..."
78435,78436,B000HDL1RQ,A2G2DXPFZKG48W,Audio Maven,6,6,5,1204070400,Great tasting snack,These are the greatest wafer type cookies I have ever eaten. I've also tried the lemon ones which are out of this world as well. They are wafer type but are cut into small cubes (ideal for snack s...
78436,78437,B000HDL1RQ,AL1DVFEY7X5VR,Always Snackin',5,5,5,1199923200,Can't eat just one!,ALL I CAN SAY IS OMG!THESE LITTLE TREATS ARE SO ADDICTIVE AND ONLY HAVE 10 GRAMS OF SUGAR IN NINE WAFERS!THESE ARE BY FAR THE BEST TASTING WAFER COOKIES I HAVE EVER HAD AND IF IT WERE NOT FOR ME B...
78437,78438,B000HDL1RQ,A2F64XFMO5ELL6,Rod B,3,3,4,1260489600,Quadratini Lemon are great!,"My wife loves these little wafers, but our local stores no longer carry them. Amazon to the rescue! Great product, fast shipping. We are very happy and will continue to order these."
78438,78439,B000HDL1RQ,A16SDTBEKKFN2Z,M. White,2,2,5,1336521600,delicious,delicious hazel nut taste in a cunchy bite- size wafer cookie. highly recommended. will buy these again.remind me of the hazel nut coffee creamer
78439,78440,B000HDL1RQ,AI47WBE7PHESF,J. Collins,2,2,5,1278720000,Punch of lemon flavor,These Loacker Quadratini Lemon Wafers are the best I've tasted. They are natural with a real lemon 'punch' in every bite. Plus they are not overly sweet like some other wafer brands. I will defini...
78440,78441,B000HDL1RQ,ALT34F4DUTJ8Z,Sassy08,2,2,5,1278288000,Bet you can't eat just one!,"Previously, these lemon wafer cookies were sold at Wal Mart; however, they no longer carry them. Recently, I discovered that Trader Joe's carried a chocolate version and I inquired if they carrie..."
78441,78442,B000HDL1RQ,A3BIN78AJ0D4JO,"MrsDiehl ""MrsDiehl""",2,2,5,1272067200,Special treat,I tried this product at a friend's house and had to get some for myself. I was delighted to find them on Amazon for half the price listed on another so called wholesale site. These make a great ...
78442,78443,B000HDL1RQ,AKFGO1HY01B85,L. Botez,2,2,5,1270598400,Yum,I love these cookies! what can I say. They are worth buying in bulk! Yum. Bet you can't eat just one.


In [6]:
#Sorting data according to ProductId in ascending order
sorted_data=df.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [7]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(393933, 10)

In [8]:
sorted_data[sorted_data.duplicated(['UserId','ProfileName',"Time","Text"])].shape

(174521, 10)

In [9]:
final.shape

(393933, 10)

In [10]:
final=final.reset_index()

In [11]:
final=final.loc[:10,:]

In [12]:
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = final['Score']
positiveNegative = actualScore.map(partition) 
final['Score'] = positiveNegative
print("Number of data points in our data", final.shape)
final.head(3)

Number of data points in our data (11, 11)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,1,1108425600,"Nice cadence, catchy rhymes",In June<br />I saw a charming group<br />of roses all begin<br />to droop<br />I pepped them up<br />with chicken soup!<br />Sprinkle once<br />sprinkle twice<br />sprinkle chicken soup<br />with ...
1,150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their months of the year! We will learn all of the poems throughout the school year. they like the handmotions which I invent for each poem.
2,150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watching the Really Rosie movie that incorporates them, and love them. My son loves them too. I do however, miss the hard cover version. The paperbacks se..."


In [13]:
from nltk.corpus import stopwords
Stopwords=stopwords.words('english')

In [14]:
for i in range(len(final['Text'])):
    words=nltk.word_tokenize(final['Text'][i])
    withoutstopwords=[word for word in words if word not in Stopwords]
    final['Text'][i]=' '.join(withoutstopwords)

C:\Users\prudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
## Remove the special characters
final['Text'] = final['Text'].str.replace("[^a-zA-Z0-9]+", " ")

In [16]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
## Remove the URL'S
import re
final['Text']=final['Text'].apply(lambda x: re.sub(r"http\S+", "", x))

In [17]:
from nltk.stem import PorterStemmer
Stemmer=PorterStemmer()
for i in range(len(final['Text'])):
    tokens=nltk.word_tokenize(final['Text'][i])
    stem_tokens=[Stemmer.stem(token) for token in tokens]
    final['Text'][i]=' '.join(stem_tokens)

C:\Users\prudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
## Remove xml tags from the reviews
from bs4 import BeautifulSoup
final['Text']=final['Text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [19]:
# remove short words (length < 3)
final['Text'] = final['Text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

In [20]:
# make entire text lowercase
final['Text'] = [r.lower() for r in final['Text']]

In [21]:
final.head(1)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,1,1108425600,"Nice cadence, catchy rhymes",june saw charm group rose begin droop pep chicken soup sprinkl sprinkl twice sprinkl chicken soup rice thi great book teach children month year the repetit phrase funni littl stori accompani pictu...


# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[10]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[20]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[30]
print(sent_4900)
print("="*50)

In [22]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [23]:
final['Text']=final['Text'].map(decontracted)

In [24]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
final['Text']=final['Text'].apply(lambda x: re.sub("\S*\d\S*", "", x))

In [25]:
final.head(2)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,1,1108425600,"Nice cadence, catchy rhymes",june saw charm group rose begin droop pep chicken soup sprinkl sprinkl twice sprinkl chicken soup rice thi great book teach children month year the repetit phrase funni littl stori accompani pictu...
1,150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,thi fun way children learn month year learn poem throughout school year like handmot invent poem


In [26]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
import re
sent_0 = re.sub("\S*\d\S*", "", "This are number 123")
print(sent_0)

This are number 


In [27]:
final.head(1)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,1,1108425600,"Nice cadence, catchy rhymes",june saw charm group rose begin droop pep chicken soup sprinkl sprinkl twice sprinkl chicken soup rice thi great book teach children month year the repetit phrase funni littl stori accompani pictu...


In [28]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in Stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 590.90it/s]


In [29]:
final['Text']=preprocessed_reviews

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(final['Text'])

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [37]:
word2tfidf

{'abl': 2.791759469228055,
 'absolut': 2.791759469228055,
 'accompani': 2.791759469228055,
 'actual': 2.791759469228055,
 'add': 2.791759469228055,
 'ago': 2.386294361119891,
 'air': 2.791759469228055,
 'along': 2.386294361119891,
 'amp': 2.791759469228055,
 'attent': 2.791759469228055,
 'beauti': 2.791759469228055,
 'bedtim': 2.791759469228055,
 'begin': 2.791759469228055,
 'bigger': 2.791759469228055,
 'bond': 2.791759469228055,
 'book': 1.2876820724517808,
 'bought': 2.791759469228055,
 'boy': 2.791759469228055,
 'brand': 2.791759469228055,
 'buy': 2.791759469228055,
 'care': 2.791759469228055,
 'carol': 2.791759469228055,
 'carri': 2.791759469228055,
 'catchi': 2.791759469228055,
 'celebr': 2.791759469228055,
 'charm': 2.791759469228055,
 'chicken': 2.386294361119891,
 'child': 2.386294361119891,
 'children': 1.8754687373538999,
 'cleaver': 2.791759469228055,
 'colleg': 2.791759469228055,
 'copi': 2.386294361119891,
 'cover': 2.791759469228055,
 'cute': 2.09861228866811,
 'daughter

In [34]:
#import warnings
# en_vectors_web_lg, which includes over 1 million unique vectors.
#import spacy
#nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()

In [35]:
import numpy as np
mean_vec1 = np.zeros([len(doc1), 96])

NameError: name 'doc1' is not defined

In [39]:
mean_vec1 = np.zeros([20, 96])
mean_vec1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(range(len(final['Text']))):
    doc1 = nlp(final['Text'][qu1]) 
    # 96 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 96])
    for i in range(len(doc1)):
        # word2vec
        word1=doc1[i]
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] = vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
#df['q1_feats_m'] = list(vecs1)


100%|██████████| 11/11 [00:00<00:00, 50.48it/s]


In [68]:
w2v_final=pd.DataFrame(vecs1)
w2v_final

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-1.361177,-2.105663,-3.277159,1.850767,1.371776,4.245814,-1.024170,3.221432,3.329229,5.875935,...,0.510548,-2.128260,-1.239677,1.152291,1.984822,0.691614,-0.370332,6.590025,2.406218,-0.528453
1,-0.967657,-0.837277,-2.274766,-0.788663,0.960751,2.508947,0.542043,1.434266,3.205433,6.216609,...,0.966807,-2.106482,-0.885439,2.109852,2.007063,1.687285,-4.691474,3.190062,2.843130,0.207812
2,-0.539028,-3.202852,-2.514946,2.016687,1.650981,3.392299,-1.332819,2.600549,2.685092,4.593647,...,-1.259637,-1.155291,-1.347253,2.453684,2.306007,0.864539,-1.380550,4.532243,2.075331,1.160882
3,1.995960,-2.188934,-5.420296,2.321700,1.690737,2.893935,0.595381,2.259609,3.053680,3.438708,...,-1.639946,-1.956266,-1.861758,1.732791,4.312851,1.669972,-1.875515,6.387399,3.201849,0.824551
4,-1.491995,-2.361095,-4.593712,0.087194,2.030077,3.155182,-1.217201,5.273329,3.068252,3.336066,...,1.221758,-1.867040,-0.956283,2.950168,3.114793,0.561583,-0.932727,5.448317,1.440686,2.431184
5,0.898631,-2.701404,-2.864704,1.861039,0.948038,4.177299,-0.810261,2.613706,1.525857,3.007959,...,1.008753,-2.407800,-1.549239,1.557871,3.084055,1.793730,-1.883566,6.405282,0.881365,1.767172
6,-0.737273,-1.785812,-2.636121,2.707965,1.662336,4.070329,-1.064514,2.236153,2.881949,3.665431,...,-0.535885,-3.225666,-0.938680,1.527443,1.555779,0.665672,-1.082395,4.314166,2.080631,0.625750
7,-0.150151,-1.610405,-1.981252,3.027184,1.793325,4.053186,-1.339714,1.844678,2.770777,5.616375,...,-0.804820,-2.121252,-1.550250,1.759327,2.399002,1.941570,-2.115822,4.075624,2.206184,2.643860
8,0.580701,-3.323321,-3.532190,3.476263,0.717794,1.415844,-0.110591,2.734948,2.408461,4.304635,...,0.100499,-1.946661,-2.728852,1.271705,0.541189,2.002827,-0.465537,3.815286,3.170541,1.078121
9,0.024966,-0.949159,-2.545496,2.507717,1.667752,2.550832,-0.440607,2.436448,2.084373,5.595065,...,-0.614420,-1.810216,-0.053640,1.923537,2.325538,0.419901,-3.078938,3.941479,2.831212,-1.224040


<h2><font color='red'>[3.2] Preprocessing Review Summary</font></h2>

In [90]:
w2v_final.head(2)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.040597,0.029095,-0.027385,0.008699,-0.000762,0.043959,-0.125500,0.049812,0.064907,0.086278,...,0.008569,0.012811,0.045857,-0.029848,-0.015161,-0.009784,-0.009713,-0.035239,0.035038,0.015609
1,0.101272,-0.016836,0.083388,-0.060804,0.253146,0.034564,-0.119985,0.092053,0.003815,-0.006391,...,0.037583,-0.011276,0.009697,-0.008541,-0.009426,0.017738,-0.003649,0.007103,0.007470,-0.008440


In [35]:
## Similartly you can do preprocessing for review summary also.

# [4] Featurization

## [4.1] WeightedW2V

In [91]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(w2v_final, final['Score'],test_size=0.3,random_state=42)

In [92]:
print('Shape of X_train is',X_train.shape)
print('Shape of y_train is',y_train.shape)

print('*'*100)

print('Shape of X_test is',X_test.shape)
print('Shape of y_test is',y_test.shape)

Shape of X_train is (7, 384)
Shape of y_train is (7,)
****************************************************************************************************
Shape of X_test is (4, 384)
Shape of y_test is (4,)


In [93]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [94]:
## Predicting the values for the X_test
predicted_xtest=list(neigh.predict(X_test))
actual_test=list(y_test)

In [95]:
## Predicting the values for the X_train
predicted_xtrain=list(neigh.predict(X_train))
actual_train=list(y_train)

In [96]:
## Passing the first value from the X_train
Predicted_value_X_train=neigh.predict(X_train.iloc[:1,:])
print('Predicted value for the first record is ',Predicted_value_X_train)

Predicted value for the first record is  [1]


In [97]:
## Passing the first value from the X_test
Predicted_value_X_test=neigh.predict(X_test.iloc[:1,:])
print('Predicted value for the first record is ',Predicted_value_X_test)

Predicted value for the first record is  [1]


In [98]:
## Passing the first value from the X_test and Predicting the Probelities
Predicted_value_X_test=neigh.predict_proba(X_test.iloc[1:20,:])
print('Predicted value for the first record is \n',Predicted_value_X_test)

Predicted value for the first record is 
 [[0.33333333 0.66666667]
 [0.33333333 0.66666667]
 [0.33333333 0.66666667]]


In [99]:
neigh.kneighbors(X_train.iloc[:1,:],return_distance=True)

(array([[0.        , 2.22819736, 2.27891552]]),
 array([[0, 6, 3]], dtype=int64))

In [100]:
neigh.kneighbors(X_train.iloc[:1,:],return_distance=False)

array([[0, 6, 3]], dtype=int64)

In [101]:
print('Classes we have to predict ',neigh.classes_)

Classes we have to predict  [0 1]


In [102]:
# Confusion metrics for the Test data

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

results =confusion_matrix(actual_test, predicted_xtest) 

print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(actual_test, predicted_xtest) )
print('Report : ')
print (classification_report(actual_test, predicted_xtest) )


Confusion Matrix :
[[0 1]
 [0 3]]
Accuracy Score : 0.75
Report : 


C:\Users\Administrator\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.75      1.00      0.86         3

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4



In [103]:
# Confusion metrics for the Test data

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

results =confusion_matrix(actual_train, predicted_xtrain) 

print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(actual_train, predicted_xtrain) )
print('Report : ')
print (classification_report(actual_train, predicted_xtrain) )

Confusion Matrix :
[[0 1]
 [0 6]]
Accuracy Score : 0.8571428571428571
Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.86      1.00      0.92         6

    accuracy                           0.86         7
   macro avg       0.43      0.50      0.46         7
weighted avg       0.73      0.86      0.79         7

